In [1]:
# Import dependencies 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# List of files to be combined
genre_files = [
    './Movie_Datasets/action.csv',
    './Movie_Datasets/adventure.csv',
    './Movie_Datasets/animation.csv',
    './Movie_Datasets/biography.csv',
    './Movie_Datasets/crime.csv',
    './Movie_Datasets/family.csv',
    './Movie_Datasets/fantasy.csv',
    './Movie_Datasets/film-noir.csv',
    './Movie_Datasets/history.csv',
    './Movie_Datasets/horror.csv',
    './Movie_Datasets/mystery.csv',
    './Movie_Datasets/romance.csv',
    './Movie_Datasets/scifi.csv',
    './Movie_Datasets/sports.csv',
    './Movie_Datasets/thriller.csv',
    './Movie_Datasets/war.csv'
]
highest_gross_file = './Movie_Datasets/Highest Holywood Grossing Movies.csv'
movies_file = './Movie_Datasets/movies.csv'

# Read in genre files
first = True
for file in genre_files:
    try:
        if first == True:
            #print('initializing df')
            genre_dfs = pd.read_csv(file)
            first = False
        else:
            #print('appending df')
            genre_df = pd.read_csv(file)
            genre_dfs = pd.concat([genre_dfs, genre_df], ignore_index=True)
    except Exception as e:
        print(f"Error reading {file}: {e}")

# Read in highest gross file
highest_gross_df = pd.read_csv(highest_gross_file)

# Read in movies file (IMDB)
movies_df = pd.read_csv(movies_file)


In [3]:
#genre_dfs.head()
#movies_df.head()
#highest_gross_df.head()

print(f"genre df has {len(genre_dfs)} records")
print(f"highest gross df has {len(highest_gross_df)} records")
print(f"movies df has {len(movies_df)} records")


genre df has 368300 records
highest gross df has 1000 records
movies df has 7668 records


In [4]:
# Rename columns
genre_dfs.rename(columns={
    'movie_name': 'title',
    'gross(in $)': 'gross',
    'certificate': 'license'
}, inplace=True)

highest_gross_df.rename(columns={
    'Title': 'title',
    'Genre': 'genre',
    'Year': 'year',
    'Release Date': 'release_date',
    'Budget (in $)': 'budget',
    'World Wide Sales (in $)': 'gross',
    'Running Time': 'runtime',
    'Distributor': 'company',
    'License': 'license'
}, inplace=True)

movies_df.rename(columns={
    'name': 'title',
    'released': 'release_date',
    'rating': 'license'
}, inplace=True)

In [5]:
# movies is the most complete dataframe, but is still missing values
# merge other dataframes into movies_df to try and fill missing values
# Eg. movies_df[movies_df['title'] == 'Dream Round'] is missing gross, but genre_dfs[genre_dfs['title'] == 'Dream Round'] has gross

# Match on title
movies_df.set_index('title', inplace=True)
genre_dfs.set_index('title', inplace=True)

# Combine_first will fill in NaN values with non-NaN values from each dataframe
merged_movies_df = movies_df.combine_first(genre_dfs)
merged_movies_df.reset_index(inplace=True)
merged_movies_df[merged_movies_df['title'] == 'Dream Round']


,title,budget,company,country,description,director,director_id,genre,gross,license,movie_id,rating,release_date,runtime,score,star,star_id,votes,writer,year
86009,Dream Round,NaN,Cactus Blue Entertainment,United States,Joe Bona struggles in life after the death of ...,Dusty Dukatz,/name/nm1184786/,Comedy,20000.0,NaN,tt9646844,5.0,"February 7, 2020 (United States)",90.0,4.7,Michael Saquella,"/name/nm1130592/,/name/nm0001298/,/name/nm9406...",36.0,Lisa Huston,2020.0


In [6]:
# Merge in highest grossing
merged_movies_df.set_index('title', inplace=True)
highest_gross_df.set_index('title', inplace=True)

# Combine_first will fill in NaN values with non-NaN values from each dataframe
merged_movies_df = merged_movies_df.combine_first(highest_gross_df)
merged_movies_df.reset_index(inplace=True)
merged_movies_df[merged_movies_df['title'] == 'Dream Round']

,title,Domestic Opening (in $),Domestic Sales (in $),International Sales (in $),Movie Info,Unnamed: 0,budget,company,country,description,...,movie_id,rating,release_date,runtime,score,star,star_id,votes,writer,year
86109,Dream Round,NaN,NaN,NaN,NaN,NaN,NaN,Cactus Blue Entertainment,United States,Joe Bona struggles in life after the death of ...,...,tt9646844,5.0,"February 7, 2020 (United States)",90.0,4.7,Michael Saquella,"/name/nm1130592/,/name/nm0001298/,/name/nm9406...",36.0,Lisa Huston,2020.0


In [7]:
# Create dataframe with only the columns we need
selected_columns = ['title', 'rating', 'genre', 'year', 'release_date', 'budget', 'director', 'star', 'gross', 'votes', 'runtime', 'company', 'license']
selected_columns = [col for col in selected_columns if col in merged_movies_df.columns]
merged_movies_df = merged_movies_df[selected_columns]
merged_movies_df['is_not_numeric'] = merged_movies_df['budget'].apply(lambda x: not isinstance(x, (int, float, np.number)))
merged_movies_df[merged_movies_df['is_not_numeric'] == True]



,title,rating,genre,year,release_date,budget,director,star,gross,votes,runtime,company,license,is_not_numeric
6578,A Little Red Flower,6.7,"Drama, Family",2020,17-Jun-11,20000000,Yan Han,"Jackson Yee, \nHaocun Liu, \nYalin Gao, \nYuan...",216197492.0,690.0,122 min,"December 31, 2020 (China)",R,True
7677,A Quiet Place Part II,7.2,"Drama, Horror, Sci-Fi",2020,7-Jun-18,May 26 2021 (Indonesia),John Krasinski,"Emily Blunt, \nMillicent Simmonds, \nCillian M...",160072261.0,237487.0,97 min,Paramount Pictures,PG-13,True
7678,A Quiet Place Part II,7.2,"Drama, Horror, Sci-Fi",2020,7-Jun-18,May 26 2021 (Indonesia),John Krasinski,"Emily Blunt, \nMillicent Simmonds, \nCillian M...",160072261.0,237497.0,97 min,Paramount Pictures,PG-13,True
7679,A Quiet Place Part II,7.2,"Drama, Horror, Sci-Fi",2020,7-Jun-18,May 26 2021 (Indonesia),John Krasinski,"Emily Blunt, \nMillicent Simmonds, \nCillian M...",160072261.0,237487.0,97 min,Paramount Pictures,PG-13,True
17893,Alvin and the Chipmunks: The Squeakquel,4.5,"Animation, Adventure, Comedy",2009,21-Dec-09,75000000,Betty Thomas,"Jason Lee, \nZachary Levi, \nDavid Cross, \nJu...",219614612.0,50645.0,88 min,Twentieth Century Fox,PG,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
366961,Youth,7.0,Comedy,2015.0,"December 4, 2015 (United States)",December 14 2017 (APAC),Paolo Sorrentino,Michael Caine,23459358.0,74000.0,124.0,Indigo Film,R,True
366962,Youth,6.5,Comedy,2015.0,"December 4, 2015 (United States)",December 14 2017 (APAC),Paolo Sorrentino,Michael Caine,23459358.0,74000.0,124.0,Indigo Film,R,True
366963,Youth,NaN,Comedy,2015.0,"December 4, 2015 (United States)",December 14 2017 (APAC),Paolo Sorrentino,Michael Caine,23459358.0,74000.0,124.0,Indigo Film,R,True
366964,Youth,NaN,Comedy,2015.0,"December 4, 2015 (United States)",December 14 2017 (APAC),Paolo Sorrentino,Michael Caine,23459358.0,74000.0,124.0,Indigo Film,R,True


In [8]:
# Remove duplicates
deduplicated_movies_df = merged_movies_df.drop_duplicates(subset='title')
print(f"deduplicated_movies_df has {len(deduplicated_movies_df)} records")
deduplicated_movies_df.dtypes

# Remove invalid budget records
deduplicated_movies_df = deduplicated_movies_df[deduplicated_movies_df['is_not_numeric'] == False]


deduplicated_movies_df has 214885 records


In [14]:
# Fix dtypes
def parse_dates(date_str):
    for fmt in ("%Y-%m-%d", "%d-%b-%Y", "%Y/%m/%d", "%B %d, %Y"):
        try:
            return pd.to_datetime(date_str, format=fmt)
        except ValueError:
            continue
    return pd.NaT  # Return NaT if none of the formats match

deduplicated_movies_df['title'] = deduplicated_movies_df['title'].astype(str)
deduplicated_movies_df['rating'] = deduplicated_movies_df['rating'].astype(float)
deduplicated_movies_df['genre'] = deduplicated_movies_df['genre'].astype(str)

deduplicated_movies_df['year'] = pd.to_numeric(deduplicated_movies_df['year'], downcast="integer", errors='coerce')
deduplicated_movies_df['year'].fillna(1800, inplace=True)
deduplicated_movies_df['year'] = deduplicated_movies_df['year'].astype(int)

deduplicated_movies_df['release_date'] = deduplicated_movies_df['release_date'].apply(parse_dates)

deduplicated_movies_df['budget'] = deduplicated_movies_df['budget'].astype(float)
deduplicated_movies_df['director'] = deduplicated_movies_df['director'].astype(str)
deduplicated_movies_df['star'] = deduplicated_movies_df['star'].astype(str)
deduplicated_movies_df['gross'] = deduplicated_movies_df['gross'].astype(float)

deduplicated_movies_df['votes'] = pd.to_numeric(deduplicated_movies_df['votes'], downcast="integer")
deduplicated_movies_df['votes'].fillna(0, inplace=True)
deduplicated_movies_df['votes'] = deduplicated_movies_df['votes'].astype(int)

deduplicated_movies_df['runtime'] = deduplicated_movies_df['runtime'].astype(str)

deduplicated_movies_df['company'] = deduplicated_movies_df['company'].astype(str)
deduplicated_movies_df['license'] = deduplicated_movies_df['license'].astype(str)
deduplicated_movies_df.dtypes

/var/folders/62/f2x4shss4kq25jc_zwqwlh740000gn/T/ipykernel_29079/4016272022.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  deduplicated_movies_df['year'].fillna(1800, inplace=True)
/var/folders/62/f2x4shss4kq25jc_zwqwlh740000gn/T/ipykernel_29079/4016272022.py:26: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting 

title                     object
rating                   float64
genre                     object
year                       int64
release_date      datetime64[ns]
budget                   float64
director                  object
star                      object
gross                    float64
votes                      int64
runtime                   object
company                   object
license                   object
is_not_numeric              bool
dtype: object

In [12]:
# Drop rows where the 'year' column has values before 1990
deduplicated_movies_df = deduplicated_movies_df[deduplicated_movies_df['year'] >= 1990]
deduplicated_movies_df

,title,rating,genre,year,release_date,budget,director,star,gross,votes,runtime,company,license,is_not_numeric
2,#1 Serial Killer,5.9,Horror,2013,NaT,NaN,Stanley Yung,"Jason Tobin, \nEugenia Yuan, \nTzi Ma, \nShosh...",NaN,62,87 min,nan,Not Rated,False
3,#1915House,NaN,"Horror, Thriller",2018,NaT,NaN,Justin DiPego,Justin DiPego,NaN,0,55 min,nan,nan,False
5,#5,5.9,"Biography, Comedy, Fantasy",2013,NaT,NaN,Ricky Bardy,"Justin Bardy, \nRicky Bardy",NaN,13,68 min,nan,nan,False
7,#66,5.6,Action,2015,NaT,NaN,Asun Mawardi,"Ari Riski Ananda, \nErwin Bagindo, \nFandy Chr...",NaN,32,116 min,nan,nan,False
8,#69 Samskar Colony,6.5,Romance,2022,NaT,NaN,Suneel Kumar Reddy,"Ajay, \nArtnilxen, \nFm Babai, \nBadram",NaN,516,126 min,nan,nan,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371071,Üç Harflilerin Musallat Oldugu Büyülü Konakta ...,2.5,"Comedy, Horror, Thriller",2018,NaT,NaN,Sinan Kaçar,"Cemre Kurum, \nEcenaz Üçer, \nOnur Durmaz, \nD...",NaN,127,80 min,nan,nan,False
371096,Üçüncü Günün Adami,8.9,Action,2017,NaT,NaN,Babak Shirinsefat,"Kamran Aghabalayev, \nNijat Ali, \nRasim Cafar...",NaN,9,nan,nan,nan,False
371102,Þriðja nafni,6.3,"Comedy, Drama, Thriller",2003,NaT,NaN,Einar Thor,"Moses Rockman, \nElma Lísa Gunnarsdóttir, \nHj...",NaN,38,88 min,nan,nan,False
371104,Καραγκιόζης the Movie,6.0,"Animation, Comedy, Family",2022,NaT,NaN,"Akis Karras, \nChristos Leivaditis, \nEkaterin...","Athos Danellis, \nElena Maragou",NaN,20,74 min,nan,nan,False


In [11]:
deduplicated_movies_df.to_csv('clean_files_output_ey.csv')